In [1]:
import pandas as pd 
import numpy as np
from  scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [2]:
df1=pd.read_csv('tmdb_5000_credits.csv')
df2=pd.read_csv('tmdb_5000_movies.csv')

df1 contains the following features:-


* movie_id - A unique identifier for each movie. 
* title - Movie title. 

* cast - The name of lead and supporting actors.

*  crew - The name of Director, Editor, Composer, Writer etc. 


df2 contains the following features:-


*   budget - The budget in which the movie was made. 


*   genre - The genre of the movie, Action, Comedy ,Thriller etc. 


*   homepage - A link to the homepage of the movie. 


*   id - This is infact the movie_id as in the first dataset. 

*   keywords - The keywords or tags related to the movie.



*   original_language - The language in which the movie was made.



*   original_title - The title of the movie before translation or adaptation.



*   overview - A brief description of the movie.


*   popularity - A numeric quantity specifying the movie popularity.


*  production_companies - The production house of the movie.


*   production_countries - The country in which it was produced.

*   release_date - The date on which it was released.


*   revenue - The worldwide revenue generated by the movie.


*   runtime - The running time of the movie in minutes.


*   status - "Released" or "Rumored".

*   tagline - Movie's tagline.

*   title - Title of the movie.

*   vote_average - average ratings the movie recieved.
*   vote_count - the count of votes recieved.





In [3]:
#Renaming df1 id column
df1.columns = ['id','tittle','cast','crew']

#Merging df1 & df2 on column 'id'
df = df2.merge(df1, on = 'id')

In [4]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [5]:
# Content Based Filtering 

Column overview has 3 Nan values. Nan values will be replaced by empty string.

In [6]:
df['overview'].isna().sum()

3

In [7]:
#Define a TF-IDF vectorizer object & remove stop words
tfidf = TfidfVectorizer(stop_words='english')

df['overview'] = df['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['overview'])

tfidf_matrix.shape

(4803, 20978)

tfidf matrix is l2 normalized. Therefore, cosine similarity is equivalent to applying linear kernel. 

In [8]:
#

#

#
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
features = ['cast', 'crew', 'keywords', 'genres','production_companies']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [10]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."


In [11]:
def get_top_five(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 5:
            names = names[:5]
        return names

    #Return empty list in case of missing data
    return []

features = ['cast', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(get_top_five)

In [12]:
def get_director(x):
    for j in x:
        if j['job'] == 'Director':
            return j['name']
    return np.nan

def get_production_companies(x):
  if isinstance(x,list):
    names = [i['name'] for i in x]
    return names 
  
  return []

df['director'] = df['crew'].apply(get_director)
df['production_companies'] = df['production_companies'].apply(get_production_companies)

Cleaning & Formatting the data

In [13]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

features = ['cast', 'keywords', 'director', 'genres','production_companies']

for feature in features:
    df[feature] = df[feature].apply(clean_data)

In [14]:
def join_together(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) + ' ' + ' '.join(x['production_companies'])
df['final'] = df.apply(join_together, axis=1)

In [28]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['final'])
count_matrix.shape

(4803, 22214)

In [16]:
count_matrix = np.array(count_matrix.toarray())
tfidf_matrix = np.array(tfidf_matrix.toarray())
final_matrix = np.hstack((count_matrix,tfidf_matrix))
final_matrix = sparse.csc_matrix(final_matrix)
final_matrix.shape

In [17]:
# Caculate Cosine similarity
cos_sim = cosine_similarity(final_matrix, final_matrix)

(4803, 43192)

In [18]:
#Reset index of the main dataframe
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

In [23]:
def recommender(title, cosine_sim=cos_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices],sim_scores

In [42]:
def recommend_movies(title):
  movies, match_percentage = recommender(title, cos_sim)
  match_percentage = [x[1] for x in match_percentage]
  match_percentage = [x*100 for x in match_percentage]
  recommendation_df = pd.DataFrame(movies.values,columns=['Movies'])
  recommendation_df['Match %'] = match_percentage
  print(recommendation_df.to_string(index=False))

In [43]:
recommend_movies('The Godfather: Part II')

                  Movies    Match %
 The Godfather: Part III  48.586173
           The Godfather  37.719581
        The Conversation  34.326496
         We're No Angels  25.928149
           The Rainmaker  23.565294
        The Untouchables  23.556938
                Scarface  23.529412
             Taxi Driver  22.992802
       The Son of No One  22.045256
                   Shaft  21.782070


In [44]:
recommend_movies('Spectre')

                                    Movies    Match %
                                   Skyfall  47.930716
                         Quantum of Solace  30.435587
                      Diamonds Are Forever  27.734835
                          Charlie's Angels  23.035129
                             Gridiron Gang  22.866478
                          Live and Let Die  22.730826
                      The Spy Who Loved Me  22.552338
                     Johnny English Reborn  22.446081
                                 RoboCop 3  22.222222
 Beastmaster 2: Through the Portal of Time  22.222222


In [47]:
recommend_movies('The Dark Knight Rises')

          Movies    Match %
 The Dark Knight  71.343666
   Batman Begins  70.721147
    The Prestige  36.979656
    Interstellar  35.000000
       Inception  34.156503
      The Losers  29.402087
    Man of Steel  29.277002
      RockNRolla  29.277002
     Harsh Times  28.867513
          Harper  28.867513
